# Język Python - Wykład 6.

## Builtin \_\_import\_\_

In [8]:
sys = __import__('sys')
sys

<module 'sys' (built-in)>

In [10]:
import importlib
importlib.import_module('os', 'path.join')

<module 'os' from '/usr/local/Cellar/python/2.7.4/Frameworks/Python.framework/Versions/2.7/lib/python2.7/os.pyc'>

In [11]:
sys.modules['os']

<module 'os' from '/usr/local/Cellar/python/2.7.4/Frameworks/Python.framework/Versions/2.7/lib/python2.7/os.pyc'>

## I/O encoding

    $ cat u.py
    # coding: utf-8
    print u"żółw"

    $ python u.py
    żółw

    $ python u.py > u.txt
    Traceback (most recent call last):
      File "u.py", line 2, in <module>
        print u"żółw"
    UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-2: ordinal not in range(128)

    $ cat u.py
    # coding: utf-8
    print u"żółw".encode('utf-8')
    
    $ python u.py > u.txt
    $ cat u.txt
    żółw

In [ ]:
print u"żółw"

In [ ]:
import sys
print sys.getdefaultencoding()

In [ ]:
sys.setdefaultencoding('utf-8')  # site.py nadpisuje moduł sys i zakrywa setdefaultencoding

In [ ]:
reload(sys)  # reload powoduje ponowne załadowanie modułu sys, site.py go nie nadpisuje 
# (warning: it's a hack!)
sys.setdefaultencoding('utf-8')  # działa

## codecs

In [ ]:
import codecs

with codecs.open('/tmp/L6', mode='w', encoding='iso-8859-2') as f:
    f.write(u'żółw')
    
with codecs.open('/tmp/L6', encoding='iso-8859-2') as f:
    string = f.readline()
    print string, repr(string)

## Chevron print

In [ ]:
import sys

print >> sys.stderr, 'Error!'
print >> sys.stdout, 'Not an error'
print >> sys.stderr, 'Error!'

## Docstring

* PEP 257

In [ ]:
class Foo:
    """Represents a Foo"""
    pass

In [ ]:
Foo.__doc__

In [ ]:
def foo_function(arg):
    """Does foo and returns False"""
    return False

In [ ]:
foo_function.__doc__

In [ ]:
def foo_multiline_doc(arg):
    """
    Does foo
    and returns 
    False
    """
    return False

Python *nie* wspiera docstringów dla zmiennych i atrybutów. 

In [ ]:
class A:
    attribute = 3
    """Attribute docstring"""
    
print A.__doc__
print A.attribute.__doc__

Jednakże niektóre systemy dokumentowania kodu (np. sphinx, epydoc) potrafią parsując kod źródłowy użyć tak skonstruowanych docstringów (warning: it's a hack!)

## "One more thing" (dekoracja)

In [ ]:
def my_decorator(f):
    def wrapper(*args, **kwds):
        print 'Calling decorated function'
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print 'Called example function'

In [ ]:
example()

In [ ]:
print example.__doc__

In [ ]:
print example.__name__

Bazinga!

In [ ]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print 'Calling decorated function'
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print 'Called example function'

In [ ]:
print example.__doc__
print example.__name__

* przy pisaniu dekoratorów **powinniśmy** dekorować funkcję dekorującą dekoratorem wraps z argumentem funkcji dekorowanej
* przypiszemy nowej funkcji wszystkie potrzebne atrybuty starej funkcji, żeby mogła ją całkowicie podmienić 

## doctest

* Powstało wiele narzędzi wykorzystujących docstringi do dziwnych rzeczy
* doctest jest modułem pythona parsującym dostringi i wykonującym testy w nich zawarte

In [ ]:
"""
Doctest dla modułu testowego


>>> sumuj(2,3)
5

OK to wszystko
"""

def sumuj(a, b):
    """
    Zwraca sumę dwóch liczb, np:
    
    >>> sumuj(1, 2)
    3
    
    >>> sumuj(1, '2')
    Traceback (most recent call last):
    ...
    TypeError: unsupported operand type(s) for +: 'int' and 'str'
    """
    return a + b

#if __name__ == "__main__":   # testujemy z iPython dlatego komentujemy tą linię
import doctest
doctest.testmod()

## assert

*  kodzie można traktować jako dodatkową dokumentację (uruchamialną)
*  nie zastępują testów (jedynie uzupełniają je)
*  pozwalają szybciej wyłapywać bugi

In [ ]:
assert True

In [ ]:
assert False, 'Something failed'

In [ ]:
assert sumuj(2, 3) == 5

Niektórzy używają assertów do kontroli typów w Pythonie! (ale to jak wiemy zabija kaczki)

In [ ]:
def sumuj(a, b):
    assert type(a) is int, "a is not an integer: %r" % a
    assert type(b) is int, "b is not an integer: %r" % b
    return a + b

In [ ]:
sumuj(3, 6)

In [ ]:
sumuj('3', 6)

Asercję można wyłączayć! (np. na produkcji ze względów wydajnościowych!)

## unittest framework

* standardowy framework testowy pythona (dostępny w modułach podstawowych pythona)
* oprócz niego istnieje jeszcze kilka ciekawych frameworków, np.:
    * nose (najpopularniejszy)
    * py.test (oparty na zwykłym poleceniu assert - dużo prostszy w użyciu niż unittest)
    http://pyvideo.org/video/645/pytest-rapid-and-simple-testing-with-python

    import unittest
    
    def sumuj(a, b):
        return a + b
    
    
    class TestSumuj(unittest.TestCase):
    
        def test_sumuj_positive(self):
            self.assertEqual(sumuj(2, 3), 5)
            
    
        def test_sumuj_negative(self):
            self.assertEqual(sumuj(2, -3), -1)
            
    
    if __name__ == '__main__':
        unittest.main()        

    class Sumator(object):
        def __cal__(self, a, b):
            return a + b

   
    class TestSumator(unittest.TestCase):
    
        def setUp(self):
            self.sumator = Sumator()
    
        def test_sumuj_positive(self):
            self.assertEqual(self.sumator(2, 3), 5)
            
        def test_sumuj_negative(self):
            self.assertEqual(self.sumator(2, -3), -1)
        
        def tearDown(self):
            del self.sumator

## mock

$ pip install mock

In [14]:
import mock
import requests 
import re
import sys

def has_title(url, keyword):
    body = requests.get(url).text
    print >> sys.stderr, body
    title = re.search(r'<title>(.*?)<\/title>', body).group(1)
    return title.lower().find(keyword) != -1
    
print has_title('http://www.onet.pl/', 'onet')

True



<!doctype html>

<html lang="pl">
<head>
    <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="application-name" content="Onet.pl"/> 
    <meta name="msapplication-TileColor" content="#ffffff"/> 
    <meta name="msapplication-TileImage" content="http://ocdn.eu/images/pulscms/ZmQ7MDA_/a3999790d747fc0b5b0b499285bb62d1.png"/>
    <meta name="HandheldFriendly" content="true" />
    <title>Onet - wersja mobilna</title>
    <link href="http://ocdn.eu/resources/template-engine/mobile.template-engine/resource/f884ee731ec5ede956fd0447deaa8788/1.103/sg_base_2.min.css" type="text/css" rel="stylesheet">
<link media="screen and (min-width:288px)" href="http://ocdn.eu/resources/template-engine/mobile.template-engine/resource/f884ee731ec5ede956fd0447deaa8788/1.103/sg_more_2.min.css" type="text/css" rel="stylesheet">
	
    <link rel="shortcut icon" href="http

In [21]:
import mock
urlmock_response = mock.MagicMock()
urlmock_response.text = u'<html><title>some Mock answer</title>'

In [16]:
urlmock.get('http://onet.pl')

<MagicMock id='4424526480'>

In [17]:
urlmock.get('http://onet.pl').text

u'<html><title>some Mock answer</title>'

In [18]:
def test_has_title():
    assert has_title('http://www.onet.pl/', 'onet') == True

test_has_title()


<!doctype html>

<html lang="pl">
<head>
    <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="application-name" content="Onet.pl"/> 
    <meta name="msapplication-TileColor" content="#ffffff"/> 
    <meta name="msapplication-TileImage" content="http://ocdn.eu/images/pulscms/ZmQ7MDA_/a3999790d747fc0b5b0b499285bb62d1.png"/>
    <meta name="HandheldFriendly" content="true" />
    <title>Onet - wersja mobilna</title>
    <link href="http://ocdn.eu/resources/template-engine/mobile.template-engine/resource/f884ee731ec5ede956fd0447deaa8788/1.103/sg_base_2.min.css" type="text/css" rel="stylesheet">
<link media="screen and (min-width:288px)" href="http://ocdn.eu/resources/template-engine/mobile.template-engine/resource/f884ee731ec5ede956fd0447deaa8788/1.103/sg_more_2.min.css" type="text/css" rel="stylesheet">
	
    <link rel="shortcut icon" href="http

In [22]:
@mock.patch('requests.get', mock.Mock(return_value=urlmock_response))
def test_has_title():
    assert has_title('http://www.onet.pl/', 'mock') == True

test_has_title()

<html><title>some Mock answer</title>
